- reference
    - https://github.com/dreamgonfly/GAN-tutorial/blob/master/GAN.ipynb

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision.utils as utils
from torchvision import datasets, transforms
import torch.nn.functional as F

is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

# 0~1로 standardize -> G에서 tanh 써야함
transform = transforms.Compose([
    transforms.ToTensor(), # change data as tensor
    transforms.Normalize(mean=[0.5], std=[0.5])]) # 0~1의 pixel값(grey scale)을 -1~1로 바꾼다

# MNIST dataset
train_data = datasets.MNIST(root='data/', train=True, transform=transform, download=True)
test_data  = datasets.MNIST(root='data/', train=False, transform=transform, download=True)


batch_size = 100
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size, shuffle=True)

In [ ]:
example_mini_batch_img, example_mini_batch_label  = next(iter(train_data_loader))

In [ ]:
dim_z = 100

In [ ]:
# input notse generator for Generator
def input_noise_generator(batch_size, dim):
    return torch.randn(batch_size, dim, device=device)

In [ ]:
# define generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(dim_z, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 28*28),
            nn.Tanh() #since normalized to -1~1
        )
    
    def forward(self, x):
        # to make visualization easy, change dimension
        return self.model(x).view(-1,1,28,28)
        

In [ ]:
# define discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.model(x)

In [ ]:
# make instance and allocate device
G = Generator()
D = Discriminator()

if is_cuda:
    G.cuda()
    D.cuda()

In [ ]:
# loss function
criterion = nn.BCELoss()

G_optimizer = optim.SGD(G.parameters(), lr = 1e-3, momentum=0.8)
D_optimizer = optim.SGD(G.parameters(), lr = 1e-3, momentum=0.8)

In [ ]:
for epoch in range(100):
    D_loss = []
    G_loss = []
    
    for mini_batch_img, mini_batch_label in train_data_loader:
        batch_size = mini_batch_img.shape[0]
        
        # convert data as tensor
        mini_batch = Variable(mini_batch_img, )
        
        # make label
        target_real = Variable(torch.ones(batch_size, 1))
        target_fake = Variable(torch.zeros(batch_size, 1))
        
        # push all tensor to cuda
        if is_cuda:
            mini_batch = mini_batch.cuda()
            target_real = target_real.cuda()
            target_fake = target_fake.cuda()
        
        D_result_real = D(mini_batch)
        
        
        
        